# Análisis preliminar del conjunto de datos

In [ ]:
# Importaciones de paquetes
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
# Importaciones de unidades de soporte
import sys
sys.path.append("..")
from src import sp_eda as sp

In [205]:
df_raw = pd.read_csv("../data/halcon_viajes_data.csv") # Leer el archivo

In [206]:
df = df_raw.copy() # Hacer una copia del dataframe con el que trabajar

In [ ]:
sp.eda_preliminar(df) # Hacer un primer análisis exploratorio de los datos

## Limpieza de datos

In [ ]:
df = df.drop(columns=['dni_usuario']) # Eliminar columnas innecesarias para el análisis

# Normalización de datos
df.columns = df.columns.str.lower()

lista_cat = ['género', 'ubicación','tipo_viajero', 'tipo_paquete','metodo_pago', 'destino',
       'tipo_alojamiento', 'clase_vuelo', 'actividades_reservadas', 'fuente_reserva','estado_reserva', 'estacionalidad',
       'comentarios']
for col in lista_cat:
  df[col] = df[col].str.replace(" ","_").str.lower()

# Convertir los datos a un formato coherente y consistente
df['género'] = df['género'].map({'female':'femenino', 'male':'masculino','other':'otro'})
df['ubicación'] = df['ubicación'].replace({'germany':'alemania', 'spain':'españa'})
df['tipo_viajero'] = df['tipo_viajero'].replace({'tusirta':'turista'})
df['tipo_paquete'] = df['tipo_paquete'].replace({'vuelo_+_hotel_+_tour': 'vuelo_hotel_tour', 'vuelo_+_hotel':'vuelo_hotel' })
df['destino'] = df['destino'].replace({'new_york':'nueva_york','london':'londres','rome':'roma','tokyo':'tokio'})

# Reemplazar True por 'si' y False por 'no' en las columnas booleanas
col_bool = ['promocion_aplicada', 'cancelacion_reserva']
df[col_bool] = df[col_bool].applymap(lambda x: 'si' if x else 'no')

# Convertir las columnas temporales a tipo datetime
for col in ['fecha_reserva', 'fecha_viaje']:
    df[col] = pd.to_datetime(df[col], format='%Y|%m|%d %H:%M:%S', errors='coerce')
    df[col] = df[col].dt.strftime('%Y/%m/%d')
    df[col] = pd.to_datetime(df[col], format='%Y/%m/%d', errors='coerce')

## Validación de datos

In [ ]:
# Validación de fechas: Asegurar que 'fecha_reserva' sea anterior a 'fecha_viaje'.
validacion_fecha = df[df['fecha_reserva'] > df['fecha_viaje']]

# Verificar que la duración del viaje es mayor que el número de noches de estancia
validacion_duracion = df[df['duracion_viaje'] <= df['numero_noches_estancia']]

# Verificar que el valor de 'mes_viaje' se corresponda con el mes de 'fecha_viaje'
validacion_mes_viaje = df[df['mes_viaje'] != df['fecha_viaje'].dt.month]

# Verificar que el valor de 'mes_reserva' se corresponda con el mes de 'fecha_reserva'
validacion_mes_reserva = df[df['mes_reserva'] != df['fecha_reserva'].dt.month]

# Verificar que las columnas 'estado_reserva' y 'cancelacion_reserva' sean coherentes
validacion_estado_cancelacion = df[(df['estado_reserva'] == 'cancelada') & (df['cancelacion_reserva'] != 'si') | (df['estado_reserva'] != 'cancelada') & (df['cancelacion_reserva'] == 'si')]

In [ ]:
# Corregir datos inconsistentes
df['mes_viaje'] = df['fecha_viaje'].dt.month
df['mes_reserva'] = df['fecha_reserva'].dt.month
# Corregir las inconsistencias entre 'estado_reserva' y 'cancelacion_reserva'
df.loc[(df['estado_reserva'] == 'cancelada') & (df['cancelacion_reserva'] != 'si'), 'cancelacion_reserva'] = 'si'
df.loc[(df['estado_reserva'] != 'cancelada') & (df['cancelacion_reserva'] == 'si'), 'cancelacion_reserva'] = 'no'

## Explicaciones de las inconsistencias detectadas en la validación de datos

### 1. Duración del viaje y número de noches de estancia

Durante el proceso de validación de datos, se observó que en **379 registros** de los datos, la **duración del viaje** es **menor o igual** al **número de noches de estancia**. Este comportamiento podría parecer ilógico, ya que se esperaría que la duración total del viaje sea siempre mayor que el número de noches pasadas en el destino.

Sin embargo, después de revisar los datos y el contexto del negocio, se concluyó que **no hay un error en los datos**. La discrepancia puede ser explicada por varias razones legítimas, como:
- Los viajeros pueden haber reservado viajes largos, pero con una estancia corta en el destino, lo que hace que la duración del viaje sea mayor que la estancia en sí.
- La duración del viaje podría incluir tiempos de vuelo, escalas o trayectos que no se cuentan como noches de estancia en el destino.

#### Decisión tomada:
Se decidió **mantener los datos tal como están**. No se realizaron modificaciones a las columnas `duracion_viaje` y `numero_noches_estancia`, ya que el comportamiento observado es plausible dentro del contexto de la gestión de viajes.

#### Impacto:
Este enfoque garantiza que no se eliminen registros ni se modifiquen datos válidos. Además, se preserva la integridad de los datos sin hacer suposiciones adicionales sobre el comportamiento de los usuarios.

### 2. Correcciones realizadas en el dataset

Durante el proceso de validación y limpieza de datos, se realizaron las siguientes correcciones para asegurar la consistencia de las columnas relacionadas con las fechas y los meses:

#### Correlación entre 'mes_viaje' y 'fecha_viaje'
Se observó que en un gran porcentaje de los datos el valor de **`mes_viaje`** no coincidía con el mes real extraído de la columna **`fecha_viaje`**. Este desajuste fue corregido de la siguiente manera:

- La columna **`mes_viaje`** fue actualizada para reflejar correctamente el mes extraído de la columna **`fecha_viaje`** utilizando la función `dt.month` de pandas.

**Impacto**: Esto asegura que la columna **`mes_viaje`** sea consistente con el mes de la fecha de viaje, mejorando la precisión de los análisis relacionados con las fechas.

#### Correlación entre 'mes_reserva' y 'fecha_reserva'
En algunos registros, el valor de **`mes_reserva`** no coincidía con el mes real extraído de la columna **`fecha_reserva`**. Para corregir esto:

- La columna **`mes_reserva`** fue actualizada para coincidir con el mes extraído de la columna **`fecha_reserva`**, utilizando la misma función `dt.month`.

**Impacto**: Este ajuste asegura que la columna **`mes_reserva`** esté alineada con el mes de la fecha de reserva, facilitando la correcta interpretación y análisis de los datos relacionados con las fechas de reserva.

### 3. Consistencia entre 'estado_reserva' y 'cancelacion_reserva'

Se identificaron algunos registros donde había incoherencias entre las dos columnas:
- En algunos casos, el estado de la reserva era **`cancelada`** pero **`cancelacion_reserva`** era **`no`**.
- En otros casos, el estado de la reserva no era **`cancelada`**, pero **`cancelacion_reserva`** era **`si`**.

#### Acción tomada:
Se corrigieron estos registros, ajustando la columna **`cancelacion_reserva`** para que refleje correctamente el estado de la reserva:
- Si la reserva fue **`cancelada`**, **`cancelacion_reserva`** se cambió a **`si`**.
- Si la reserva no fue **`cancelada`**, **`cancelacion_reserva`** se cambió a **`no`**.

#### Impacto:
Este ajuste garantiza que las columnas **`estado_reserva`** y **`cancelacion_reserva`** sean consistentes, lo que mejora la calidad de los datos para los análisis posteriores.

## Gestión de valores atípicos

In [233]:
# Creamos un dataframe con las columnas numericas
columnas_num = df.select_dtypes(include = np.number).columns.to_list()
lista_eliminar = [ 'mes_reserva', 'mes_viaje']
for col in lista_eliminar:
  columnas_num.remove(col)

In [ ]:
# Observamos las medidas estadísticas
df[columnas_num].describe().T

In [ ]:
# Comprobamos la presencia de valores atípicos comparando los histogramas con los diagramas de cajas de cada columna
sp.plot_outliers(df, columnas_num )

En base a los análisis visuales realizados, **no se han identificado valores atípicos** en el dataset. Esto indica que los datos parecen estar bien distribuidos y dentro de rangos razonables.

## Gestión de nulos

In [ ]:
df.isnull().sum()

Se observaron algunos **valores nulos** en tres columnas del dataset. Estos valores nulos tienen un **sentido lógico** y **no indican errores** en la recopilación de datos, sino que reflejan casos específicos dentro del contexto del negocio.

**Columna 'comentarios'**: Algunos usuarios pueden no dejar comentarios, ya que no todos los clientes proporcionan retroalimentación sobre su experiencia.

**Columna 'tipo_alojamiento' y 'actividades_reservadas' para 'Solo vuelo'**: Se espera que para 'solo vuelo' no haya valores en las columnas **`tipo_alojamiento`** o **`actividades_reservadas`**.

In [236]:
# Rellenar los nulos con valores fijos
df['tipo_alojamiento'] = df['tipo_alojamiento'].fillna('Sin alojamiento')
df['actividades_reservadas'] = df['actividades_reservadas'].fillna('Sin actividad') 
df['comentarios'] = df['comentarios'].fillna('Sin comentarios')

## Creación de nuevas columnas interesantes para el análisis

In [ ]:
# Crear gasto por persona y gasto por duracion del viaje
df['gasto_por_persona'] = round(df['costo_total'] / df['numero_personas'], 2)
df['gasto_por_duracion'] = round(df['costo_total'] / df['duracion_viaje'],2)
# Crear una columna de rango de edad
bins = [0, 18, 30, 40, 50, 60, 100]
labels = ['0-18', '19-30', '31-40', '41-50', '51-60', '60+']
df['rango_edad'] = pd.cut(df['edad'], bins=bins, labels=labels, right=False)
# Verificar las nuevas columnas
df[['edad','gasto_por_persona', 'gasto_por_duracion', 'rango_edad']].sample(10)

In [246]:
df.to_csv("../data/halcon_viajes_limpios.csv", index = False)